# Import packages 

In [1]:
import pandas as pd
import numpy as np

# Read the .csv

In [15]:
df_job = pd.read_csv('cleaned_datajob.csv', encoding='utf-8-sig')
df_job.head()

,Job_Title,Location,Job_Highlight,JRJD,Company_Name
0,ASSISTANT MANAGER - DATA ANALYTICS,Wan Chai,Min 5-7 years’ experience in data anal...,[' Requirement s Bachelor d...,Sun Hung Kai Properties
1,IT Assistant (Data Management),Mong Kok,"5-days work, bank holiday', 'Data ad mi...",[' Job Responsibilities : To...,EIDEALINK LIMITED
2,"Associate Vice President, Dashboard Reporting ...",Wan Chai,"Degree in Statistics, Business Analysi...",[' Responsibilities Direct report ...,United Asia Finance Limited
3,Data Center Customer Operations Engineer / Fie...,Tsuen Wan Area,"Open to Fresh Graduate applicants', 'O...",[' Who are we? Equinix is one of t...,Equinix Hong Kong Ltd
4,IT Data Centre Operator,Tsuen Wan Area,"Monitor & manage data center facilities', ' ...",[' What you’ll be doing? Perfo...,OCBC Wing Hang Bank Limited


In [16]:
lst_jrjd = df_job.JRJD.tolist()
lst_jobhighligh = df_job.Job_Highlight.tolist()

In [4]:
lst_jrjd[39]

'["    We would like to invite undergraduates who are passionate to develop their future careers in the financial services industry to participate in our Summer Internship Program.    The purpose of our program is to provide students with an opportunity to gain hands-on work experience in the corporate environment.     If you believe you are capable, motivated, and deliver, we want you to join our AGBA family this Summer. Program Period:        Jun – Aug 2023    Responsibilities    :    Provide ad    ministrative support to product team in implementing product strategies of GI, Life Insurance and MPF products;    Support product team to have the product due diligence, product comparison, update product information, product -launch, market analysis, etc.;    Support product team in preparing presentation and sales collateral;    Assist in B2C digital platform development;    Participate in system development project on User Acceptance Test (UAT) to support rollout of Core Wealth Managem

# NLP Process for job highlight

In [5]:
import spacy
import re

## Entity Ruler For Extracting Experience information 

In [27]:
config = {
   "overwrite_ents": True,
   "validate" : True
}

nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe('entity_ruler', before="ner", config=config)
#ruler = nlp.add_pipe('entity_ruler', after="ner", config=config)
#ruler = nlp.add_pipe('entity_ruler', config=config)

# Min 5-7 year
# 1-2 years working experience
# 10+ years

min_lst = ["min", "min.", "minimum"]



pattern_exp = [
    {"label" : "EXPERIENCE",
     "pattern" : [ {"LOWER": {"IN": min_lst}},
                   {"IS_DIGIT" : True},
                   {"IS_PUNCT" : True, "OP" : "?"},
                   {"IS_DIGIT" : True, "OP" : "?"},
                   {"LEMMA" : "year", "LOWER" : "years","OP" : "+"}]},

    {"label" : "EXPERIENCE",
     "pattern" : [ {"LOWER": {"IN": min_lst}, "OP" : "?"},
                   {"IS_ALPHA" : True, "OP" : "?"},
                   {"IS_DIGIT" : True}, 
                   {"IS_PUNCT" : True, "OP" : "?"},
                   {"IS_DIGIT" : True, "OP" : "?"},
                   {"IS_SPACE" : True, "OP" : "?"},
                   {"IS_SPACE" : True, "OP" : "?"},
                   {"LEMMA" : "year", "LOWER" : "years","OP" : "+"},
                   {"LEMMA" : "of", "LOWER" : "of","OP" : "?"},
                   {"LEMMA" : "work", "LOWER" : "work","OP" : "?"},
                   {"LEMMA" : "experience", "LOWER" : "experience","OP" : "?"}]},

    {"label" : "EXPERIENCE",
     "pattern" : [ {"IS_DIGIT" : True},
                   {"TEXT" : "+", "OP" : "?"},
                   {"IS_SPACE" : True, "OP" : "?"},
                   {"IS_SPACE" : True, "OP" : "?"},
                   {"LEMMA" : "year", "LOWER" : "years","OP" : "+"},
                   {"LEMMA" : "work", "LOWER" : "work","OP" : "?"},
                   {"LEMMA" : "experience", "LOWER" : "experience","OP" : "?"}]},
                
    {"label" : "EXPERIENCE",
     "pattern" : [ {"LOWER" : "at", "OP" : "?"},
                   {"LOWER" : "least", "OP" : "+"},
                   {"IS_DIGIT" : True},
                   {"LEMMA" : "year", "LOWER" : "years","OP" : "+"},
                   {"POS" : "ADJ", "OP" : "?"},
                   {"LEMMA" : "work", "LOWER" : "work","OP" : "?"},
                   {"LEMMA" : "experience", "LOWER" : "experience","OP" : "?"}]},

    # 1 year working experience
    {"label" : "EXPERIENCE",
     "pattern" : [ {"IS_DIGIT" : True},
                   {"LOWER" : "year", "LEMMA" : "year" },
                   {"LOWER" : "working"},
                   {"LOWER" : "experience"} ]},

    {"label" : "EXPERIENCE",
     "pattern" : [ 
         {"lower" : "at"},
         {"lower" : "least"},
         {"IS_DIGIT" : True},
         {'lower' : 'year'},
         {"POS" : "ADJ"},
         {"POS" : "NOUN"}
      ]}

    ]

ruler.add_patterns(pattern_exp)

In [28]:
text = 'Minimum of 5 years'
nlp(text)

ent_text = nlp(text)

for ent in ent_text.ents:
    #if ent.label_ == "EXPERIENCE":
    #    print(ent.text, ent.label_)
    print(ent.text, ent.label_)

for item in ent_text:
    print(item.pos_)

Minimum of 5 years EXPERIENCE
NOUN
ADP
NUM
NOUN


In [29]:
text = lst_jrjd[58]
nlp(text)

ent_text = nlp(text)

for ent in ent_text.ents:
    #if ent.label_ == "EXPERIENCE":
    #    print(ent.text, ent.label_)
    print(ent.text, ent.label_)

ETL ORG
Data Visualization ORG
Data Modelling ORG
Dashboard BuildingJob PERSON
Preferably 2 years of EXPERIENCE
Data Analysis ORG
Business Intelligence ORG
SQL ORG
English LANGUAGE
Chinese NORP
Graduates LOC
Kerry Logistics Network Company Webpage ORG
CV PERSON
Group ORG
Human Resources & Ad ORG
Kerry Logistics PERSON
2614 DATE
the Kerry Group ORG


## Matcher For if the position is open to FG

In [14]:
from spacy.matcher import Matcher

In [15]:
matcher = Matcher(nlp.vocab)

lst_fresh = ['fresh']
lst_grad = ['grad', 'graduate', 'graduates', 'student','students']

# Welcome FG
pattern_fg = [
    {"LEMMA" : "welcome"},
    {"LOWER" : "to", "OP" : "?"},
    {"LOWER": {"IN": lst_fresh}},
    {"LOWER": {"IN": lst_grad}},
]

matcher.add("ACCEPT_FG", [pattern_fg])

# Open to FG
pattern_fg = [
    {"LEMMA" : "open"},
    { "LEMMA" : "to"},
    {"LOWER": {"IN": lst_fresh}},
    {"LOWER": {"IN": lst_grad}},
]

matcher.add("ACCEPT_FG", [pattern_fg])

# FG is welcome
pattern_fg = [
    {"LOWER" : {"IN" : lst_fresh} },
    {"LOWER" :  {"IN" : lst_grad} },
    {"LOWER" : "is"},
    {"LOWER" : "welcome"}

]

matcher.add("ACCEPT_FG", [pattern_fg])

pattern_fg = [
    {"LOWER" : "fresh" },
    {"IS_SPACE" : True, "OP":"*"},
    {"LOWER" :  {"IN" : lst_grad}}
]

matcher.add("ACCEPT_FG", [pattern_fg], greedy="LONGEST")

# FG 
pattern_fg = [
    {"LOWER" : "fg" }
]

matcher.add("ACCEPT_FG", [pattern_fg], greedy="LONGEST")

In [46]:
text = lst_jobhighligh[4]
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])

for match in matches:
    print (doc[match[1]:match[2]])
    print (match)



## Capture the education requirement using Entity Ruler

In [53]:
min_lst = ["min", "min.", "minimium"]  

# "Prefix"
#xx Science degree 
lst_pre = ['computer', 'data', 'management', 'information', 'quantitative']
#Business
lst_suf = ['technology', 'science', 'analysis', 'system', 'engineering', 'systems']

lst_alone = ['finance', 'accounting', 'mis', 'engineering', 'telecom', 'logistics', 'marketing', 'economics']

# This is for matching the exact term since without captalizing is confusing (Captalized is Major, Without is a skill)
lst_exact = ['Business', 'Data', 'Supply']
lst_analy = ['Analysis', 'Analytics', 'Chain', 'Management']

# "Suffix"


pattern_edulv = [
    {"label" : "EDU_LV",
     "pattern" : [ {"LOWER": "bachelor"},
                  {"LOWER" : "degree"}]},

    {"label" : "EDU_LV",
     "pattern" : [ {"LOWER": "master"}]},

     # Ph.D.
    {"label" : "EDU_LV",
     "pattern" : [ {"LOWER": "ph"},
                  {"IS_PUNCT" : True},
                  {"LOWER" : "d"},
                  {"IS_PUNCT" : True}]},

    # University degree holder
    {"label" : "EDU_LV",
     "pattern" : [ {"LOWER": "university", "OP" : "?"},
                  {"LOWER" : "degree"},
                  {"LOWER" : "holder", "LEMMA" : "holder", "OP" : "?"}]}
]

ruler.add_patterns(pattern_edulv)

pattern_major = [
    {"label" : "MAJOR",
     "pattern" : [ {"LOWER": {"IN": lst_pre}},
                   {"LOWER" : {"IN" : lst_suf}}]},
    
    {"label" : "MAJOR",
     "pattern" : [ {"LOWER": "statistics"}]},

    {"label" : "MAJOR",
     "pattern" : [ {"LOWER": "it"}]},

    # Business Analysis / Data Analytics must be 
    {"label" : "MAJOR",
     "pattern" : [ {"TEXT" : {"IN": lst_exact}} , 
                  {"TEXT" : {"IN": lst_analy}} ]},

    {"label" : "MAJOR",
     "pattern" : [ {"LOWER" : {"IN" : lst_alone} } ]},


    {"label" : "MAJOR",
     "pattern" : [ {"TEXT" : "Business" } ]},

    ]

ruler.add_patterns(pattern_major)

In [52]:
lst_jrjd[26]

"['       Responsibilities       :       Responsible for regular and efficient tenant database maintenance, mapping and enhancement, and work closely with Business Operation teams for data cleansing and consolidationDrive enhancement of business data structure and data integration across different platforms with the use of technologyAutomate business and data process to decrease manual workflowAssist Team Manager to        oversee execution of data management and analytics by working closely with IT department and other data analytics teams in system enhancement projects        Assist in budgeting, forecasting, planning and analytics projects as assignedRequirement       s :              Degree in Information Systems, Business or related discipline       s At least 3-5 years of solid experience in real estate/       retail industryStrong business acumen and excellent communication skillsProficient in computer applications including Salesforce, Tableau, PowerBI, Oracle, and other Micros

In [54]:
text = lst_jrjd[26]
nlp(text)

ent_text = nlp(text)

for ent in ent_text.ents:
    #if ent.label_ == "EXPERIENCE":
    #    print(ent.text, ent.label_)
    print(ent.text, ent.label_)

'       Responsibilities       :       Responsible WORK_OF_ART
Business MAJOR
IT MAJOR
Degree EDU_LV
Information Systems MAJOR
Business MAJOR
least 3-5 years of EXPERIENCE
Salesforce GPE
Tableau GPE
Oracle ORG
Microsoft ORG
Excel PRODUCT
Word PERSON
English LANGUAGE
Chinese NORP
Business MAJOR
Business MAJOR
Human Resources Department ORG
Apply Now”        button. WORK_OF_ART
the Hang Lung Group ORG
8 weeks DATE
12 months DATE
'Responsibilities       :       ' WORK_OF_ART
'Requirement       s :       ']        WORK_OF_ART


In [21]:
text = "Supply Chain, Logistics,Statistics"
nlp(text)

ent_text = nlp(text)
for ent in ent_text.ents:
    #if ent.label_ == "EXPERIENCE":
    #    print(ent.text, ent.label_)
    print(ent.text, ent.label_)

Supply Chain MAJOR
Logistics MAJOR
Statistics MAJOR


In [ ]:
lst_jrjd[61:90]

## Capture required skill

In [ ]:
# SAS, Oracle
# SQL, MSSQL, MS SQL , PostgreSQL, MySQL
# BI tools / Power BI / Tableau
# Microsoft Azure Cloud, Azure, AWS, Microsoft Azure Data, Excel 
# Python, R, Snowflake 
# Mongo DB 

In [ ]:
str = "Data Mining\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0"

test = str.replace("\xa0", '')

print(test)